In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
metadata_index = 27

In [3]:
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()

In [4]:
df = pd.read_excel("JSE587U.xlsx",  header=None) # sheetname=sheet,
df.dropna(how="all", inplace=True) 

In [5]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,Software Version,2.06.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Experiment File Path:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Protocol File Path:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Plate Number,Plate 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Date,2018-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Time,09:33:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Reader Type:,Synergy H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Reader Serial Number:,1411074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Reading Type,Reader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Procedure Details,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# above row 31 is metadata
# below is plate

# could make dictionary of dictionary for Procedure Details, Results?
# Maybe....
testdic = {}
for i,j in zip(df[:metadata_index][0], df[:metadata_index][1]):
    if(i not in testdic.keys() and not pd.isna(i) ):
        key = i
        testdic[key] = []
    testdic[key].append(j)

In [7]:
print(testdic)

{'Software Version': ['2.06.10'], 'Experiment File Path:': [nan], 'Protocol File Path:': [nan], 'Plate Number': ['Plate 1'], 'Date': [datetime.datetime(2018, 10, 31, 0, 0)], 'Time': [datetime.time(9, 33, 35)], 'Reader Type:': ['Synergy H1'], 'Reader Serial Number:': [1411074], 'Reading Type': ['Reader'], 'Procedure Details': [nan], 'Plate Type': ['96 WELL PLATE'], 'Read': ['Absorbance Endpoint', 'Full Plate', 'Wavelengths:  600', 'Read Speed: Normal,  Delay: 100 msec,  Measurements/Data Point: 8', 'Fluorescence Endpoint', 'Full Plate', 'Filter Set 1', '    Excitation: 485,  Emission: 515', '    Optics: Top,  Gain: 60', 'Light Source: Xenon Flash,  Lamp Energy: High', 'Read Speed: Normal,  Delay: 100 msec,  Measurements/Data Point: 10', 'Read Height: 7 mm'], 'Results': [nan], 'Actual Temperature:': [35, 34.7, nan]}


In [8]:
import json

with open("testdata.json", "w") as test:
    json.dump(testdic, test, indent=4, sort_keys=True, default = myconverter)

##r = json.dumps(testdic, default = myconverter, ) 

In [9]:
plate = df[metadata_index:]
plate = plate.dropna(axis=1,how='all')
plate = plate.dropna(axis=0,how='all')
plate = plate.reset_index(drop=True)
plate_normalized_plate = plate.iloc[::2]
plate_normalized_plate = plate_normalized_plate.set_index(1)
plate_normalized_plate.index.name = 'Index'
for i in plate_normalized_plate.columns:
    plate_normalized_plate.rename(columns={ i : str(i-1) },inplace=True)
plate_normalized_plate = plate_normalized_plate.reset_index()
plate_not_normalized_plate = plate[1:].iloc[::2]
plate_not_normalized_plate = plate_not_normalized_plate.set_index(1)
plate_not_normalized_plate.index.name = 'Index'
for i in plate_not_normalized_plate.columns:
    plate_not_normalized_plate.rename(columns={ i : str(i-1) },inplace=True)
plate_not_normalized_plate = plate_not_normalized_plate.reset_index()
plate_not_normalized_plate['Index'] = plate_normalized_plate['Index'] 

In [10]:
plate_normalized_plate = plate_normalized_plate.melt(id_vars=['Index'],value_name="Value")

In [11]:
plate_normalized_plate["PlatemapID"] = plate_normalized_plate["Index"]+plate_normalized_plate["variable"].astype(str)
plate_normalized_plate.head()

,Index,variable,Value,PlatemapID
0,A,1,0.826,A1
1,B,1,0.892,B1
2,C,1,0.905,C1
3,D,1,0.939,D1
4,E,1,0.932,E1


In [12]:
plate_normalized_plate.rename(columns={'Value' : 'Read 1:600'},inplace=True)

In [13]:
plate_not_normalized_plate = plate_not_normalized_plate.melt(id_vars=['Index'],value_name="Value")
plate_not_normalized_plate["PlatemapID"] = plate_not_normalized_plate["Index"]+plate_not_normalized_plate["variable"].astype(str)
plate_not_normalized_plate.rename(columns={'Value' : 'Read 2:485,515'},inplace=True)
plate_not_normalized_plate.head()

,Index,variable,"Read 2:485,515",PlatemapID
0,A,1,2705,A1
1,B,1,2013,B1
2,C,1,1898,C1
3,D,1,1784,D1
4,E,1,1667,E1


In [14]:
plate_not_normalized_plate["Plate_Number"] = testdic["Plate Number"][0]

In [15]:
plate_normalized_plate.head()

,Index,variable,Read 1:600,PlatemapID
0,A,1,0.826,A1
1,B,1,0.892,B1
2,C,1,0.905,C1
3,D,1,0.939,D1
4,E,1,0.932,E1


In [16]:
plate_reformattet = pd.merge(right=plate_not_normalized_plate, right_on="PlatemapID", left=plate_normalized_plate, left_on="PlatemapID")

In [18]:
plate_reformattet = plate_reformattet[['Read 1:600', 'PlatemapID','Read 2:485,515', 'Plate_Number']]

In [20]:
plate_reformattet.to_excel("20190730_platereader_reformat.xlsx",index=False)